In [13]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

import nltk
from nltk.tokenize import word_tokenize

In [14]:
newData = pd.read_csv('sephora_products_dataset.csv')
newData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    259 non-null    int64  
 1   Label         259 non-null    object 
 2   URL           259 non-null    object 
 3   brand_name    259 non-null    object 
 4   product_name  259 non-null    object 
 5   description   259 non-null    object 
 6   price         259 non-null    object 
 7   score         259 non-null    float64
 8   skin_type     259 non-null    object 
 9   skin_concern  257 non-null    object 
 10  ingredients   256 non-null    object 
 11  image_url     208 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage: 24.4+ KB


In [15]:
newData.Label[newData['Label'] == 'face-wash-for-men'] = str('Face Wash')
newData.Label[newData['Label'] == 'moisturizer-men'] = str('Moisturizer')
newData.Label[newData['Label'] == 'mens-grooming-kits-mens-shaving'] = str('Shaving Kits')
newData.Label[newData['Label'] == 'mens-fragrance'] = str('Fragrance')
newData.Label[newData['Label'] == 'eye-cream-men'] = str('Eye Cream')
newData.Label[newData['Label'] == 'deodorant-for-men-body-spray'] = str('Deodorant')
newData.Label[newData['Label'] == 'mens-hair-products'] = str('Hair products')

In [16]:
#create product id column
newData.rename(columns={'Unnamed: 0':'product_id'}, inplace=True)
newData

,product_id,Label,URL,brand_name,product_name,description,price,score,skin_type,skin_concern,ingredients,image_url
0,0,Face Wash,https://www.sephora.com/product/soy-face-clean...,Fresh,Soy Makeup Removing Face Wash,A bestselling three-in-one face wash that gent...,$38.00,4.4,"Normal, Dry, Combination, and Oily","Dryness, Dullness and Uneven Texture, and Loss...","Water, Coco-Glucoside, Glycerin, Butylene Glyc...",https://www.sephora.com/productimages/sku/s487...
1,1,Face Wash,https://www.sephora.com/product/daily-microfol...,Dermalogica,Daily Microfoliant Exfoliator,"A gentle, rice-based powder exfoliant that vis...",$59.00,4.6,"Normal, Dry, Combination, and Oily\n","Fine Lines and Wrinkles, Pores, and Uneven Tex...","Microcrystalline Cellulose, Magnesium Oxide, S...",https://www.sephora.com/productimages/sku/s200...
2,2,Face Wash,https://www.sephora.com/product/special-cleans...,Dermalogica,Special Cleansing Gel,A soap-free cleanser that gently removes impur...,$39.00,4.4,"Normal, Dry, Combination, and Oily\n","Pores, Dryness, and Oiliness","Water/Aqua/Eau, Disodium Laureth Sulfosuccinat...",https://www.sephora.com/productimages/sku/s200...
3,3,Face Wash,https://www.sephora.com/product/mini-daily-mic...,Dermalogica,Mini Daily Microfoliant Exfoliator,"A gentle, rice-based powder exfoliant that vis...",$15.00,4.3,"Normal, Dry, Combination, and Oily","Fine Lines and Wrinkles, Pores, and Uneven Tex...","Microcrystalline Cellulose, Magnesium Oxide, S...",https://www.sephora.com/productimages/sku/s222...
4,4,Face Wash,https://www.sephora.com/product/facial-fuel-en...,Kiehl's Since 1851,Facial Fuel Energizing Face Wash,"A Kiehl’s favorite, this men’s face cleanser r...",$24.00,4.8,"Normal, Dry, Combination, and Oily","Dullness, Uneven Texture, and Puffiness","Water, Glycerin, Ammonium Lauryl Sulfate, Prop...",https://www.sephora.com/productimages/sku/s198...
...,...,...,...,...,...,...,...,...,...,...,...,...
254,254,Hair products,https://www.sephora.com/product/hold-it-togeth...,SEPHORA COLLECTION,Hold It Together: No-Slip Bobby Pins,What it is: A set of 45 non-slip bobby pins in...,$5.00,4.3,"Straight, Wavy, Curly, and Coily","Frizz, Hold, and Shine",No Info,https://www.sephora.com/productimages/sku/s216...
255,255,Hair products,https://www.sephora.com/product/grooming-solut...,Kiehl's Since 1851,Grooming Solutions Clean Styling Gel,"A non-greasy, flake-free styling gel that prov...",$18.00,2.0,"Straight ,Wavy ,Curly ,Coiled, Tightly Coiled","Frizz, Hold, and Shine","Water, Propylene Glycol, Pvp, Peg-14 Dimethico...",https://www.sephora.com/productimages/sku/s198...
256,256,Hair products,https://www.sephora.com/product/grooming-solut...,Kiehl's Since 1851,Grooming Solutions Texturizing Clay,A styling clay that shapes and texturizes hair...,$18.00,3.0,"Straight ,Wavy ,Curly ,Coiled,Tightly Coiled","Frizz, Hold, and Shine",-Eucalyptus Oil: Soothes skin while providing ...,https://www.sephora.com/productimages/sku/s198...
257,257,Hair products,https://www.sephora.com/product/esperienza-2-v...,Rossano Ferretti Parma,Esperienza #2 Volume Regime,A volume regime travel kit that adds increased...,$59.00,4.4,"Straight ,Wavy ,Curly ,Coiled ,Tightly Coiled","Frizz, Hold, and Shine","Water, Laureth-5 Carboxylic Acid, Disodiumlaur...",https://www.sephora.com/productimages/sku/s184...


In [17]:
# skin type

newData['skin_type'] = newData['skin_type'].str.replace('and','')
newData['skin_type'] = newData['skin_type'].str.replace('\\n','')

newData['skin_type']

0                     Normal, Dry, Combination,  Oily
1                     Normal, Dry, Combination,  Oily
2                     Normal, Dry, Combination,  Oily
3                     Normal, Dry, Combination,  Oily
4                     Normal, Dry, Combination,  Oily
                            ...                      
254                     Straight, Wavy, Curly,  Coily
255     Straight ,Wavy ,Curly ,Coiled, Tightly Coiled
256      Straight ,Wavy ,Curly ,Coiled,Tightly Coiled
257     Straight ,Wavy ,Curly ,Coiled ,Tightly Coiled
258    Straight ,Wavy, Curly , Coiled ,Tightly Coiled
Name: skin_type, Length: 259, dtype: object

In [18]:
# convert skin type categorical variables into dummy variables

pattern = re.compile(r"([a-zA-Z]+)")
for i in range(len(newData)):
    newData['skin_type'][i] = re.findall(pattern, newData['skin_type'][i])

## list column dummies
df_2 = newData['skin_type'].str.join('|').str.get_dummies()
newData= newData.join(df_2).drop('skin_type', axis = 1)

newData.head()

,product_id,Label,URL,brand_name,product_name,description,price,score,skin_concern,ingredients,...,Coily,Combination,Curly,Dry,Normal,Oily,Sensitive,Straight,Tightly,Wavy
0,0,Face Wash,https://www.sephora.com/product/soy-face-clean...,Fresh,Soy Makeup Removing Face Wash,A bestselling three-in-one face wash that gent...,$38.00,4.4,"Dryness, Dullness and Uneven Texture, and Loss...","Water, Coco-Glucoside, Glycerin, Butylene Glyc...",...,0,1,0,1,1,1,0,0,0,0
1,1,Face Wash,https://www.sephora.com/product/daily-microfol...,Dermalogica,Daily Microfoliant Exfoliator,"A gentle, rice-based powder exfoliant that vis...",$59.00,4.6,"Fine Lines and Wrinkles, Pores, and Uneven Tex...","Microcrystalline Cellulose, Magnesium Oxide, S...",...,0,1,0,1,1,1,0,0,0,0
2,2,Face Wash,https://www.sephora.com/product/special-cleans...,Dermalogica,Special Cleansing Gel,A soap-free cleanser that gently removes impur...,$39.00,4.4,"Pores, Dryness, and Oiliness","Water/Aqua/Eau, Disodium Laureth Sulfosuccinat...",...,0,1,0,1,1,1,0,0,0,0
3,3,Face Wash,https://www.sephora.com/product/mini-daily-mic...,Dermalogica,Mini Daily Microfoliant Exfoliator,"A gentle, rice-based powder exfoliant that vis...",$15.00,4.3,"Fine Lines and Wrinkles, Pores, and Uneven Tex...","Microcrystalline Cellulose, Magnesium Oxide, S...",...,0,1,0,1,1,1,0,0,0,0
4,4,Face Wash,https://www.sephora.com/product/facial-fuel-en...,Kiehl's Since 1851,Facial Fuel Energizing Face Wash,"A Kiehl’s favorite, this men’s face cleanser r...",$24.00,4.8,"Dullness, Uneven Texture, and Puffiness","Water, Glycerin, Ammonium Lauryl Sulfate, Prop...",...,0,1,0,1,1,1,0,0,0,0


In [19]:
# skin concern 
newData['skin_concern'] = newData['skin_concern'].str.replace('and','')

In [20]:
# ingridients

newData = newData.loc[pd.notnull(newData['ingredients'])]

#tokenize ingredients
newData['ingredients'] = newData['ingredients'].apply(word_tokenize)
newData['ingredients']

0      [Water, ,, Coco-Glucoside, ,, Glycerin, ,, But...
1      [Microcrystalline, Cellulose, ,, Magnesium, Ox...
2      [Water/Aqua/Eau, ,, Disodium, Laureth, Sulfosu...
3      [Microcrystalline, Cellulose, ,, Magnesium, Ox...
4      [Water, ,, Glycerin, ,, Ammonium, Lauryl, Sulf...
                             ...                        
254                                           [No, Info]
255    [Water, ,, Propylene, Glycol, ,, Pvp, ,, Peg-1...
256    [-Eucalyptus, Oil, :, Soothes, skin, while, pr...
257    [Water, ,, Laureth-5, Carboxylic, Acid, ,, Dis...
258    [Water, ,, Disodium, Laureth, Sulfosuccinate, ...
Name: ingredients, Length: 256, dtype: object

In [21]:
pattern = ['\r\n', '-\w+: ', 'Please', 'No Info', 'This product', 'Visit']

for i in range(len(newData)):
    Num = len(a[i])
    for j in range(Num):
        if all(x not in a[i][j] for x in pattern):
            newData['ingredients'][i] = a[i][j]

In [22]:
newData.to_csv('cleaned_sephora_dataset.csv', encoding = 'utf-8-sig', index = False)